# PLS-DA for Ralstonia

## Imports

In [ ]:
import os
import datetime 

import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cross_decomposition import PLSRegression, CCA, PLSSVD

import plotly.graph_objects as go
import plotly.express as px

pd.options.plotting.backend = "plotly"

## Functions

In [ ]:
def scatter_model(model, df, x_new, column_names, loadings=None, title=""):
    pc1_lbl = f"PC1 ({model.explained_variance_ratio_[0] * 100:.2f}%)"
    pc2_lbl = f"PC2 ({model.explained_variance_ratio_[1] * 100:.2f}%)"
    x = x_new[:, 0]
    y = x_new[:, 1]

    df[pc1_lbl] = x * (1.0 / (x.max() - x.min()))
    df[pc2_lbl] = y * (1.0 / (y.max() - y.min()))

    fig = px.scatter(
        data_frame=df,
        x=pc1_lbl,
        y=pc2_lbl,
        color="disease_idx_str",
        title=title,
    )    
    if loadings is not None:
        m = 1 / np.amax(loadings)
        loadings = loadings * m
        xc, yc = [], []
        for i in range(loadings.shape[0]):
            xc.extend([0, loadings[i, 0], None])
            yc.extend([0, loadings[i, 1], None])
        fig.add_trace(
            go.Scatter(
                x=xc,
                y=yc,
                mode="lines",
                name="Loadings",
                showlegend=False,
                line=dict(color="black"),
                opacity=0.3,
            )
        )
        fig.add_trace(
            go.Scatter(
                x=loadings[:, 0],
                y=loadings[:, 1],
                mode="text",
                text=column_names,
                opacity=0.7,
                name="Loadings",
            ),
        )

    fig.update_layout(
        height=800,
        title=title,
    )

    return fig

In [ ]:
def plot_variance(df_ev):
    df_ev = df_ev.assign(cumulative=df_ev["exp_var_per"].cumsum())
    ev_fig = go.Figure()
    ev_fig.add_trace(
        go.Bar(
            x=df_ev["pc"],
            y=df_ev["exp_var_per"],
            name="individual",
        )
    )
    ev_fig.add_trace(
        go.Scatter(
            x=df_ev["pc"],
            y=df_ev["cumulative"],
            name="cumulative",
        )
    )
    ev_fig.update_layout(
        height=800,
        title="Explained variance by different principal components",
        xaxis_title="Principal component",
        yaxis_title="Explained variance in percent",
    )
    return ev_fig

## Load dataframe

In [ ]:
df_src = pd.read_csv(
    os.path.join("..", "data_in", "020s1804_nem_v5.1.csv")
).drop(
    ["date_time", "experiment", "plant_id", "biol_rep"],
    axis=1,
).assign(
    day_after_start=lambda x: x.day_after_start.astype(int)
).sort_values(
    ["plant", "date"]
)
df_src = df_src[df_src.date != datetime.date(year=2018,month=4,day=19)]
df_src


In [ ]:
df_src.sort_values(
    [
        "disease_index", 
        "plant", 
        "date"
    ]
).select_dtypes(
    np.number
).apply(lambda x: x / x.max()).drop(
    ["day_after_start"], 
    axis=1,
).reset_index(
    drop=True
).assign(
    disease_index=lambda x: x.disease_index * 4
).plot(
    kind="line",
    height=800,
    facet_col="disease_index",
)


In [ ]:
df_src.date.plot.bar()

In [ ]:
df_src.isnull().sum().sum()

In [ ]:
sgl_val = []
for column in df_src.columns:
    if len(df_src[column].unique()) < 2:
        sgl_val.append(column)

sgl_val


## Prepare dataframe

In [ ]:
X = (
    df_src.drop(
        [
            "day_after_start",
            "disease_index",
        ],
        axis=1,
    ).select_dtypes(np.number)
)

column_names = X.columns.to_list()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

## PCA

### Fit PCA

In [ ]:
pca_data = PCA()
x_new = pca_data.fit_transform(X)


### Plot

In [ ]:
scatter_model(
    model=pca_data,
    df=df_src.copy(),
    x_new=x_new,
    column_names=column_names,
    loadings=np.transpose(pca_data.components_[0:2, :]),
    title="PCA with loadings"
)


In [ ]:
plot_variance(
    df_ev=pd.DataFrame.from_dict(
        {
            "pc": [f"PC{i}" for i in range(len(pca_data.explained_variance_ratio_))],
            "exp_var_per": pca_data.explained_variance_ratio_ * 100,
        }
    )
)


## PLS-DA

#### Find the optimal components

##### Brute force

In [ ]:
best = (0,0)
pls_data = None
for i in range(2, X.shape[1] + 1):
    pls_tmp = PLSRegression(n_components=i)
    x_new = pls_tmp.fit(
        X, 
        df_src.disease_index,
    ).transform(X)
    score = pls_tmp.score(X, df_src.disease_index)
    if score > best[1]:
        best = (i, score, pls_tmp)

best

##### Guided by correlation

In [ ]:
corr_matrix = df_src.drop(["day_after_start"], axis=1).corr()

In [ ]:
px.imshow(corr_matrix, text_auto=True, height=1400)

In [ ]:

for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    best_columns = (
        corr_matrix[
            (corr_matrix.disease_index > threshold)
            | (corr_matrix.disease_index < -threshold)
        ][["disease_index"]]
        .sort_values(["disease_index"])
        .index.to_list()
    )
    best_columns.remove("disease_index")
    if best_columns:
        Xc = df_src[best_columns]
        pls_data = PLSRegression(n_components=Xc.shape[1])
        x_new = pls_data.fit(
            Xc,
            df_src.disease_index,
        ).transform(Xc)
        score = pls_data.score(Xc, df_src.disease_index)
        # best.append(())
        if score > best[1]:
            best = (threshold, score)
best


### Fit

In [ ]:
pls_data = PLSRegression(n_components=47)
x_new = pls_data.fit(
    X, 
    df_src.disease_index,
).transform(X)

In [ ]:
pls_data.score(X, df_src.disease_index)

### Plots

In [ ]:
px.scatter(
    x=pls_data.x_scores_[:,0] / pls_data.x_scores_[:,0].max(),
    y=pls_data.x_scores_[:,1] / pls_data.x_scores_[:,1].max(),
    color=df_src.disease_idx_str,
    height=800,
)

In [ ]:
px.scatter_3d(
    x=pls_data.x_scores_[:,0] / pls_data.x_scores_[:,0].max(),
    y=pls_data.x_scores_[:,1] / pls_data.x_scores_[:,1].max(),
    z=pls_data.x_scores_[:,2] / pls_data.x_scores_[:,2].max(),
    color=df_src.disease_idx_str,
    height=800,
)